## Carga de Datos y Generación de Embeddings

En este notebook, se lleva a cabo la carga de un conjunto de datos y la generación de embeddings utilizando un modelo preentrenado de `SentenceTransformers`. Estos embeddings se almacenan en un índice de Pinecone, lo que permite una búsqueda y recuperación eficiente. A continuación, se describen las operaciones realizadas:

1. **Importación de Librerías**:
   - Se importan las bibliotecas necesarias:
     - `pandas` para la manipulación de datos.
     - `SentenceTransformer` de la biblioteca `sentence_transformers` para generar embeddings a partir de texto.
     - `Pinecone` para interactuar con el servicio de Pinecone.
     - `os` y `dotenv` para manejar variables de entorno y cargar la configuración necesaria.

2. **Carga de Variables de Entorno**:
   - Se utiliza `load_dotenv()` para cargar las variables de entorno definidas en un archivo `.env`, permitiendo acceder a credenciales sensibles, como la clave API de Pinecone.

3. **Carga del DataFrame**:
   - Se carga un archivo CSV llamado **'series.csv'** utilizando `pd.read_csv()`. Este DataFrame contiene los datos de las series que se utilizarán para generar los embeddings.

4. **Creación de una Instancia de Pinecone**:
   - Se obtiene la clave API utilizando `os.getenv("key")`, que autentica la conexión con el servicio de Pinecone.
   - Se crea una instancia de `Pinecone` usando la clave API, y se conecta al índice **'series'** para poder insertar los embeddings generados.

5. **Carga del Modelo de Embeddings**:
   - Se carga un modelo de embeddings llamado **'all-MiniLM-L6-v2'** de `SentenceTransformers`, que es un modelo ligero y eficiente para la generación de embeddings a partir de texto.

6. **Generación de Embeddings**:
   - Los embeddings se generan a partir de la columna **'embedding'** del DataFrame utilizando el modelo cargado. Los embeddings se convierten en una lista mediante `tolist()`.

7. **Inserción de Embeddings en Pinecone**:
   - Se itera sobre los embeddings generados, y cada embedding se inserta en Pinecone utilizando el método `upsert()`. Aquí, se almacena cada vector junto con su **IMDb ID** correspondiente, lo que permite una recuperación fácil y efectiva de los datos en el futuro.

8. **Confirmación de Carga**:
   - Finalmente, se imprime un mensaje de confirmación indicando que los vectores han sido cargados en Pinecone, lo que señala el éxito de la operación.

Este proceso es fundamental para habilitar un sistema de búsqueda eficiente y escalable, facilitando la recuperación de información basada en el contenido de las series.


In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv()
# Crea una instancia de Pinecone
API_key = os.getenv("key")


# Cargar el DataFrame
df = pd.read_csv("../data/clean_data/series.csv") 
# Crea una instancia de Pinecone
pc = Pinecone(api_key= API_key) 
index = pc.Index('series')
# Cargar modelo de embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')  # Modelo ligero para generar embeddings

embeddings = model.encode(df['embedding'].tolist())

# Insertar embeddings en Pinecone
for i, embedding in enumerate(embeddings):
    index.upsert([(df['IMDb ID'][i], embedding.tolist())]) # Almacena el vector en Pinecone con el ID del libro
print("Vectores cargados en Pinecone.")

/opt/anaconda3/envs/env_finalp/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Vectores cargados en Pinecone.
